In [1]:
# === Required Packages Installation ===
!pip install pdfplumber gradio transformers sentencepiece speechrecognition pyaudio pydub --quiet
!apt-get install -y portaudio19-dev python3-all-dev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
ERROR: ERROR: Failed 

In [2]:
!pip install SpeechRecognition
!pip install pyaudio
!pip install pdfplumber

  Using cached speechrecognition-3.14.3-py3-none-any.whl.metadata (30 kB)
Using cached speechrecognition-3.14.3-py3-none-any.whl (32.9 MB)
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp311-cp311-linux_x86_64.whl size=67423 sha256=f02821dd19477f0ef077994ac717abf8cb6970bc4972e5c5b4baa29d221eedfe
  Stored in directory: /root/.cache/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built pyaudio
  Using cached pdfplumber-0.11.7-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer_six-20250506-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.7-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20250506-py3-none-any.whl (5.6 MB)
Using cached pypdfium

In [3]:
# === Imports ===
from transformers import pipeline
import gradio as gr
import speech_recognition as sr
from pydub import AudioSegment
import pdfplumber
import tempfile
import os
import re

In [4]:
# === Load FLAN-T5 Model ===
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [5]:
# === Resume Text Chunking ===
def split_resume(resume_text, max_len=512):
    words = resume_text.split()
    chunks, chunk = [], []

    for word in words:
        chunk.append(word)
        if len(" ".join(chunk)) > max_len:
            chunks.append(" ".join(chunk[:-1]))
            chunk = [word]

    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

In [6]:
# === Extract Text from PDF ===
def extract_text_from_pdf(pdf_file):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()

In [7]:
# === Question Generator ===
def generate_questions_from_resume(resume_text, num_questions=5):
    resume_chunks = split_resume(resume_text)
    questions = []

    for chunk in resume_chunks:
        result = qa_pipeline(
            f"Generate {num_questions} technical interview questions based on the following resume:\n{chunk}",
            max_length=256,
            do_sample=True,
            temperature=0.7,
        )
        generated_text = result[0]['generated_text']
        extracted = re.findall(r"[\-•]?\s*(.*?)(?:\?|$)", generated_text)
        questions += [q.strip("•-–1234567890. \n") for q in extracted if q.strip()]

    return questions[:num_questions]

In [8]:
# === Audio Transcription ===
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    sound = AudioSegment.from_file(audio_file)

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpfile:
        temp_wav = tmpfile.name
        sound.export(temp_wav, format="wav")

    try:
        with sr.AudioFile(temp_wav) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        text = "Could not understand the audio."
    except sr.RequestError:
        text = "Speech recognition service error."
    finally:
        os.remove(temp_wav)

    return text

In [9]:
# === Answer Scoring ===
def score_answer_with_context(answer, resume, question):
    prompt = f"""You are an interview evaluator.

Resume:
{resume}

Question:
{question}

Candidate's Answer:
{answer}

Evaluate the answer on a scale of 1 to 10 (only return a number):"""

    result = qa_pipeline(prompt, max_new_tokens=10)[0]['generated_text']

    try:
        score = int(''.join(filter(str.isdigit, result.strip())))
        score = min(max(score, 1), 10)
    except:
        score = "N/A"
    return str(score)

In [10]:
# === Gradio App ===
with gr.Blocks() as app:
    gr.Markdown("# AI Interviewer with Voice Response and PDF Resume")

    with gr.Row():
        resume_pdf = gr.File(label="📄 Upload Resume (PDF Only)", type="filepath")
        resume_text_output = gr.Textbox(label="📃 Extracted Resume Text", lines=10)

    generate_btn = gr.Button("Generate Interview Questions")
    question_list = gr.Dropdown(choices=[], label="Pick a Question")

    mic_input = gr.Audio(type="filepath", label="Record Your Answer")
    evaluate_btn = gr.Button("Evaluate Answer")

    transcript_output = gr.Textbox(label="Transcript of Your Answer")
    feedback_output = gr.Textbox(label="Score (1-10)")

    questions_holder = gr.State([])
    resume_holder = gr.State("")

    def extract_and_store(pdf_path):
        text = extract_text_from_pdf(pdf_path)
        return text, text

    def generate_questions(resume):
        questions = generate_questions_from_resume(resume)
        return gr.update(choices=questions, value=questions[0] if questions else None), questions

    def evaluate(audio, question, resume, questions):
        if not question:
            return "Please select a question.", "N/A"
        transcript = transcribe_audio(audio)
        score = score_answer_with_context(transcript, resume, question)
        return transcript, score

    resume_pdf.change(extract_and_store, inputs=resume_pdf, outputs=[resume_text_output, resume_holder])
    generate_btn.click(generate_questions, inputs=resume_holder, outputs=[question_list, questions_holder])
    evaluate_btn.click(evaluate, inputs=[mic_input, question_list, resume_holder, questions_holder],
                       outputs=[transcript_output, feedback_output])

app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd8e1ee2268fe1ca5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
